In [2]:
import numpy as np
# from matplotlib import pyplot as plt
# import pymc3 as pm

In [3]:
class CascadingModelEnv():
	def __init__(self, E: list[float]):
		self.E = E

	def click(self, A: list[int]) -> int:
		for i in A:
			if self.E[i] > np.random.random():
				return i
		return -1

In [25]:
## test
E = [0.1, 0, 0, 0.2, 0.3, 0, 0.2, 0.3, 0, 0.1]
cme = CascadingModelEnv(E)
cme.click([0,3,4])

3